In [6]:
import pandas as pd
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")
low_memory=False
import os

path = os.path.join(os.getcwd()+"/Output")

if not (os.path.exists(path)):
    os.makedirs(path)
csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith(".csv"):
        csv_files.append(file)

for files in csv_files:
    df2=pd.read_csv(files)
    df3 = pd.read_csv('Source/BANKNIFTY-I.csv')

    for col in df3.columns[2:]:
        df3[col].values[:] = 0  

    df3['Date'] = pd.to_datetime(df3['Date'],format="%Y/%m/%d")
    df3['Time'] = pd.to_datetime(df3['Time'],format="%H:%M:%S").dt.time

    df2['Date'] = pd.to_datetime(df2['Date'],format="%Y/%m/%d")
    df2['Time'] = pd.to_datetime(df2['Time'],format="%H:%M:%S").dt.time

    df3["DT"] = df3['Date'].astype(str) + ' ' + df3['Time'].astype(str)

    df2["DT"] = df2['Date'].astype(str) + ' ' + df2['Time'].astype(str)

    r1 = df2[(pd.to_datetime(df2['DT'])).isin(pd.to_datetime(df3['DT']))]

    r2 = df3[~(pd.to_datetime(df3['DT'])).isin(pd.to_datetime(df2['DT']))]

    df = pd.concat([r2,r1],axis=0,ignore_index = True)

    df = df.sort_values(by="DT",ignore_index = True)

    data = pd.read_csv("Source/OPTIDX_BANKNIFTY_CE_02-09-2020_TO_01-09-2022.csv")

    expiry = data["Expiry"]
    expiry = pd.to_datetime(expiry)
    expiry = pd.to_datetime(expiry,format="%Y/%m/%d")    

    expiry_date = expiry[expiry.isin(df["Date"])]

    expiry_date.drop_duplicates(inplace=True)

    expiry_date.sort_values(ignore_index=True,inplace=True)

    df["Expiry"] = 0

    es = df["Date"].isin(expiry)

    for i in range(len(df)-1):
        if ((es[i] == True) & (df["Time"][i]=='15:30:59')):
            df["Expiry"][i+1] = 1

    for i in range(len(df)-1):
        if ((df["Expiry"][i+1]==0) & (df["Open"][i+1]==0)): 
            df["Open"][i+1]=df["Close"][i]
            df['High'][i+1] = df['Close'][i]
            df['Low'][i+1] = df['Close'][i]
            df['Close'][i+1] = df['Close'][i] 
            df['Open Interest'][i+1] = df['Open Interest'][i]

    df = df.drop(['DT', 'Expiry'], axis=1)
    df.to_csv(f"Output/{files}",index=False)

KeyboardInterrupt: 